In [1]:
import networkx as nx
import numpy as np
import scipy.sparse as ssp
from argparse import ArgumentParser

## Reading from the graph

In [2]:
def _lines_in_file(filename):
    with open(filename) as f:
        for i, _ in enumerate(f):
            pass
    return i + 1


In [3]:
def graph_reader(filename):
    graph_data = []
    with open(filename) as f:
        for i, line in enumerate(f):
            vals = line.split(' ')
            if len(vals) == 4:
                vals = vals[:-1]
            elif len(vals) == 3:
                vals[-1] = vals[-1][:-1]
            else:
                raise ValueError("Incorrect format in line {}".format(i + 1))
            vals = [int(v) for v in vals]
            graph_data.append(tuple(vals))

    assert len(graph_data) == _lines_in_file(p.graph_file),"Incorrect file read - input file has {} lines ""while only {} edges have been added".format(_lines_in_file(p.graph_file), len(graph_data))

    G = nx.DiGraph()  # directed graph
    G.add_weighted_edges_from(graph_data)

    return G

## Defining graph parameters

In [4]:
def get_adjacency_matrix(graph_obj, meta_data=False):
    """
    This function returns a n x n adjacency matrix given a networkx graph object.

    Arguments:
        graph_obj : Networkx graph object
        meta_data : Argument to toggle to get meta_data of graph optionally. Default=False

    Returns:
        If meta_data is True, then a 2-tuple : (adjacency matrix, meta-data dictionary)
        If meta_data is False, then a 2-tuple : (adjacency matrix, None)
        Please note adjacency matrix returned is in sparse format. Use `.todense()` to convert
        to dense format
    """
    adj_mat = nx.adjacency_matrix(graph_obj)  # this is a sparse matrix
    metas = None
    if meta_data:
        metas = {}
        metas['density'] = adj_mat.getnnz() / (adj_mat.shape[0] * adj_mat.shape[1])
        metas['positive'] = (adj_mat > 0).getnnz()
        metas['negative'] = (adj_mat < 0).getnnz()
        metas['links'] = adj_mat.getnnz()
        metas['average_links'] = adj_mat.getnnz() / adj_mat.shape[0]
    return (adj_mat, metas)

In [5]:
def get_absolute_adjacency_matrix(graph_obj):
    """
    This function returns a n x n adjacency matrix given a networkx graph object, where the
    entries are absolute values of the weights between the edges

    Arguments:
        graph_obj : Networkx graph object

    Returns:
        n x n absolute adjacency matrix (sparse)
    """
    adj_mat, _ = get_adjacency_matrix(graph_obj, False)
    adj_mat = abs(adj_mat)
    return adj_mat


In [6]:
def get_symmetric_adjacency_matrix(graph_obj):
    """
    This function returns a n x n given a networkx graph object, which is the sum of the
    adjacency matrix and its transpose. This provides symmetry.

    Arguments:
        graph_obj : Networkx graph object

    Returns:
        n x n matrix (sparse)
    """
    adj_mat, _ = get_adjacency_matrix(graph_obj, False)
    adj_mat_t = adj_mat.transpose()
    return adj_mat + adj_mat_t

In [7]:
def get_absolute_symmetric_adjacency_matrix(graph_obj):
    """
    This function returns a n x n given a networkx graph object, which is the sum of the
    absolute adjacency matrix and its transpose. This provides symmetry.

    Arguments:
        graph_obj : Networkx graph object

    Returns:
        n x n matrix (sparse)
    """
    abs_adj_mat = get_absolute_adjacency_matrix(graph_obj)
    abs_adj_mat_t = abs_adj_mat.transpose()
    return abs_adj_mat + abs_adj_mat_t

In [8]:
def get_absolute_diagonal_degree_matrix(graph_obj):
    """
    This function returns a diagonal matrix D which is the defined as
    Dii = sum |Aij| for all j

    Arguments:
        graph_obj : Networkx graph object

    Returns:
        n x n matrix (sparse)
    """
    abs_adj_mat = get_absolute_adjacency_matrix(graph_obj)
    diag_entries = np.array(abs_adj_mat.sum(axis=1)).reshape(-1)
    return ssp.coo_matrix(np.diag(diag_entries))

In [9]:
def get_absolute_symmetric_diagonal_degree_matrix(graph_obj):
    """
    This function returns a diagonal matrix E which is the defined as
    Eii = sum |Bij| for all j

    Arguments:
        graph_obj : Networkx graph object

    Returns:
        n x n matrix (sparse)
    """
    abs_sym_adj_mat = get_absolute_symmetric_adjacency_matrix(graph_obj)
    diag_entries = np.array(abs_sym_adj_mat.sum(axis=1)).reshape(-1)
    return ssp.coo_matrix(np.diag(diag_entries))

## Finding clustering coefficients 

In [20]:
def clustering_coeff(graph_obj):
    abs_adj_mat = get_absolute_adjacency_matrix(graph_obj)
    sq_abs_adj_mat = abs_adj_mat.dot(abs_adj_mat)
    return abs_adj_mat.multiply(sq_abs_adj_mat).sum() / sq_abs_adj_mat.sum()


In [22]:
def sign_clustering_coeff(graph_obj):
    adj_mat, metas = get_adjacency_matrix(graph_obj)
    sq_adj_mat = adj_mat.dot(adj_mat)
    abs_adj_mat = abs(adj_mat)
    sq_abs_adj_mat = abs_adj_mat.dot(abs_adj_mat)
    return adj_mat.multiply(sq_adj_mat).sum() / sq_abs_adj_mat.sum()


In [18]:
def relative_sign_clustering_coeff(graph_obj):
    C = clustering_coeff(graph_obj)
    C_s = sign_clustering_coeff(graph_obj)
    return C_s / C

## Main function

In [13]:
if __name__ == '__main__':
    p = ArgumentParser()
    p.add_argument('--graph_file', type=str, required=True, help='File location to get graph in trivial graph format\
                                     <node1>[space]<node2>[space]<weight>')
    p = p.parse_args("--graph_file ../slashdot-zoo/out.matrix".split())
    G = graph_reader(p.graph_file)

In [23]:
rel_sign_coeff = relative_sign_clustering_coeff(G)

In [24]:
rel_sign_coeff

0.8270075744535788